In [146]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from ydata_profiling import ProfileReport
import numpy as np
import pandas as pd
import dic_moledo
from pyspark.sql.functions import col, create_map, lit
from itertools import chain
import importlib

importlib.reload(dic_moledo)

<module 'dic_moledo' from 'c:\\Users\\mathe\\OneDrive\\Documentos\\Códigos\\RECONHECIMENTO-DE-PADROES-COM-INTELIGENCIA-ARTIFICIAL-MICRODADOS-ENEM\\dic_moledo.py'>

In [147]:
# Configura a SparkSession
spark = SparkSession.builder \
    .appName("Minha Aplicação") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.executor.memoryOverhead", "1g") \
    .getOrCreate()

# Agora você pode verificar todas as configurações atuais
conf = spark.sparkContext.getConf()
print({item[0]: item[1] for item in conf.getAll()})



{'spark.app.submitTime': '1713161579874', 'spark.driver.extraJavaOptions': '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false', 'spark.driver.memory': '4g', 'spark.executor.memory': '4g', 'spark

In [148]:
# Lendo um arquivo CSV
df = spark.read.csv("MICRODADOS_ENEM_2022.csv", header=True, sep=";")

In [149]:
colunas_descartadas = ['NU_INSCRICAO', 'NU_ANO', 'TP_ENSINO','CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 
                       'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT','TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 
                       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT', 'NU_NOTA_COMP1', 
                       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5','TP_SEXO','TP_ESTADO_CIVIL','Q011','Q005','Q007','Q008','Q009','Q010','Q012','Q013','Q015',
                       'Q016','Q017','Q018','Q019','Q020','Q021','Q023','Q014','Q022','Q025','TP_ST_CONCLUSAO','TP_FAIXA_ETARIA','IN_TREINEIRO']

# Descartando colunas
df = df.drop(*colunas_descartadas)

df.show()

+-----------+----------------+---------------+---------+-----------+-----------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+-----------------+---------------+----+----+----+----+----+----+
|TP_COR_RACA|TP_NACIONALIDADE|TP_ANO_CONCLUIU|TP_ESCOLA|CO_UF_PROVA|SG_UF_PROVA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|TP_LINGUA|TP_STATUS_REDACAO|NU_NOTA_REDACAO|Q001|Q002|Q003|Q004|Q006|Q024|
+-----------+----------------+---------------+---------+-----------+-----------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------+-----------------+---------------+----+----+----+----+----+----+
|          2|               1|              2|        1|         53|         DF|             0|             0|             0|             0|      NULL|      NULL|      NULL|      NULL|        0|             NULL|           

In [150]:
# Descartando linhas que contem valores diferentes de 1 em TP_PRESENCA_CN
df = df.filter(df.TP_PRESENCA_CN == 1)
df = df.filter(df.TP_PRESENCA_CH == 1)
df = df.filter(df.TP_PRESENCA_LC == 1)
df = df.filter(df.TP_PRESENCA_MT == 1)

df.count()

# Descartando colunas que contem valores H na coluna Q001 e Q002
df = df.filter(df.Q001 != 'H')
df = df.filter(df.Q002 != 'H')

df.count()

2122885

In [151]:
# Criando a coluna media e descartando as colunas de notas
df = df.withColumn('NU_NOTA_MEDIA', (col('NU_NOTA_CN') + col('NU_NOTA_CH') + col('NU_NOTA_LC') + col('NU_NOTA_MT') + col('NU_NOTA_REDACAO'))/5)
df = df.drop('NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT')

df.show()

+-----------+----------------+---------------+---------+-----------+-----------+--------------+--------------+--------------+--------------+---------+-----------------+---------------+----+----+----+----+----+----+------------------+
|TP_COR_RACA|TP_NACIONALIDADE|TP_ANO_CONCLUIU|TP_ESCOLA|CO_UF_PROVA|SG_UF_PROVA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|TP_LINGUA|TP_STATUS_REDACAO|NU_NOTA_REDACAO|Q001|Q002|Q003|Q004|Q006|Q024|     NU_NOTA_MEDIA|
+-----------+----------------+---------------+---------+-----------+-----------+--------------+--------------+--------------+--------------+---------+-----------------+---------------+----+----+----+----+----+----+------------------+
|          2|               1|              2|        1|         29|         BA|             1|             1|             1|             1|        1|                1|            760|   E|   F|   A|   D|   B|   A|            558.24|
|          3|               1|              2|        1|        

In [152]:
colunas_descartadas = ['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT','TP_STATUS_REDACAO','SG_UF_PROVA','NU_NOTA_REDACAO','TP_ANO_CONCLUIU','TP_NACIONALIDADE']

# Descartando colunas
df = df.drop(*colunas_descartadas)


In [153]:
print(dic_moledo.q006_dict)

{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17}


In [154]:
# Colunas Q001 ate Q025
colunas = ['Q001','Q002','Q003','Q004','Q006','Q024']


for coluna in colunas:
    # Pegando os valores correspondentes a cada letra no dicionario dic
    nome_dict = coluna.lower() + '_dict'  # Constrói o nome do dicionário
    if hasattr(dic_moledo, nome_dict):  # Verifica se o dicionário existe no módulo dicionario
        mapeamento = getattr(dic_moledo, nome_dict)  # Obtém o dicionário
        mapeamento_expr = create_map([lit(x) for x in chain(*mapeamento.items())])
        df = df.withColumn(coluna, mapeamento_expr.getItem(col(coluna)))


C:\Users\mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\sql\column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [155]:
df.show()

+-----------+---------+-----------+---------+----+----+----+----+----+----+------------------+
|TP_COR_RACA|TP_ESCOLA|CO_UF_PROVA|TP_LINGUA|Q001|Q002|Q003|Q004|Q006|Q024|     NU_NOTA_MEDIA|
+-----------+---------+-----------+---------+----+----+----+----+----+----+------------------+
|          2|        1|         29|        1|   5|   6|   1|   4|   2|   1|            558.24|
|          3|        1|         32|        1|   3|   1|   1|   2|   1|   2|            394.62|
|          2|        1|         33|        1|   5|   5|   2|   2|   2|   1|             414.1|
|          3|        1|         26|        1|   5|   5|   2|   2|   2|   1|             438.1|
|          1|        3|         28|        1|   5|   5|   2|   1|   2|   1|             576.7|
|          3|        1|         35|        0|   7|   6|   6|   6|   2|   2|            530.58|
|          2|        1|         31|        1|   2|   6|   1|   4|   2|   1|             645.8|
|          1|        1|         33|        1|   2|

In [156]:
# Substituindo valores da CO_UF_PROVA para valores de regiao
# Regiao Suldeste = 1
# Regiao Sul = 2
# Regiao Centro-Oeste = 3
# Regiao Nordeste = 4
# Regiao Norte = 5
df = df.withColumn("CO_UF_PROVA", 
    when(col("CO_UF_PROVA").isin(['11', '12', '13', '14', '15', '16', '17']), 5)
    .when(col("CO_UF_PROVA").isin(['21', '22', '23', '24', '25', '26', '27', '28', '29', '30']), 4)
    .when(col("CO_UF_PROVA").isin(['50', '51', '52', '53']), 3)
    .when(col("CO_UF_PROVA").isin(['41', '42', '43']), 2)
    .when(col("CO_UF_PROVA").isin(['31', '32', '33', '35']), 1)
    # Se nenhum dos when se aplicar, mantém o valor original da coluna 'CO_UF_PROVA'
    .otherwise(col("CO_UF_PROVA"))
)
df.show()


+-----------+---------+-----------+---------+----+----+----+----+----+----+------------------+
|TP_COR_RACA|TP_ESCOLA|CO_UF_PROVA|TP_LINGUA|Q001|Q002|Q003|Q004|Q006|Q024|     NU_NOTA_MEDIA|
+-----------+---------+-----------+---------+----+----+----+----+----+----+------------------+
|          2|        1|          4|        1|   5|   6|   1|   4|   2|   1|            558.24|
|          3|        1|          1|        1|   3|   1|   1|   2|   1|   2|            394.62|
|          2|        1|          1|        1|   5|   5|   2|   2|   2|   1|             414.1|
|          3|        1|          4|        1|   5|   5|   2|   2|   2|   1|             438.1|
|          1|        3|          4|        1|   5|   5|   2|   1|   2|   1|             576.7|
|          3|        1|          1|        0|   7|   6|   6|   6|   2|   2|            530.58|
|          2|        1|          1|        1|   2|   6|   1|   4|   2|   1|             645.8|
|          1|        1|          1|        1|   2|

In [157]:
df.describe()
df = df.withColumn("CO_UF_PROVA", df["CO_UF_PROVA"].cast("int"))
df.describe()


DataFrame[summary: string, TP_COR_RACA: string, TP_ESCOLA: string, CO_UF_PROVA: string, TP_LINGUA: string, Q001: string, Q002: string, Q003: string, Q004: string, Q006: string, Q024: string, NU_NOTA_MEDIA: string]

In [158]:
# Alterando o tipo de dado de string para int 
df = df.withColumn("TP_COR_RACA", df["TP_COR_RACA"].cast("int"))
df = df.withColumn("TP_ESCOLA", df["TP_ESCOLA"].cast("int"))
df = df.withColumn("TP_LINGUA", df["TP_LINGUA"].cast("int"))

In [159]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# Convert the Spark DataFrame back to a pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()
result_pdf.dtypes

C:\Users\mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\sql\pandas\conversion.py:111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 4.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


TP_COR_RACA        int32
TP_ESCOLA          int32
CO_UF_PROVA        int32
TP_LINGUA          int32
Q001               int32
Q002               int32
Q003               int32
Q004               int32
Q006               int32
Q024               int32
NU_NOTA_MEDIA    float64
dtype: object

In [122]:
'''result_pdf = pd.get_dummies(result_pdf)
result_pdf.head()'''

'result_pdf = pd.get_dummies(result_pdf)\nresult_pdf.head()'

In [123]:
'''profile = ProfileReport(result_pdf, title="Pandas Profiling Report")
profile.to_file("output_q006_reload.html")'''

'profile = ProfileReport(result_pdf, title="Pandas Profiling Report")\nprofile.to_file("output_q006_reload.html")'

In [124]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [140]:
df_teste = df

# Calcula os quantis aproximados para a coluna 'MEDIAS'
limite_baixo = df_teste.stat.approxQuantile('NU_NOTA_MEDIA', [0.33], 0.01)[0]
limite_alto = df_teste.stat.approxQuantile('NU_NOTA_MEDIA', [0.66], 0.01)[0]

# Classifica cada linha em 'baixo', 'médio', 'alto' com base nos quantis calculados
df_teste = df.withColumn(
    'CLASSE_MEDIAS',
    when(col('NU_NOTA_MEDIA') <= limite_baixo, 'baixo')
    .when(col('NU_NOTA_MEDIA') <= limite_alto, 'médio')
    .otherwise('alto')
)

# Mostra o DataFrame com a nova coluna de classificação
df_teste.show()

+-----------+---------+-----------+---------+----+----+----+----+----+----+------------------+-------------+
|TP_COR_RACA|TP_ESCOLA|CO_UF_PROVA|TP_LINGUA|Q001|Q002|Q003|Q004|Q006|Q024|     NU_NOTA_MEDIA|CLASSE_MEDIAS|
+-----------+---------+-----------+---------+----+----+----+----+----+----+------------------+-------------+
|          2|        1|          4|        1|   5|   6|   1|   4|   2|   1|            558.24|        médio|
|          3|        1|          1|        1|   3|   1|   1|   2|   1|   2|            394.62|        baixo|
|          2|        1|          1|        1|   5|   5|   2|   2|   2|   1|             414.1|        baixo|
|          3|        1|          4|        1|   5|   5|   2|   2|   2|   1|             438.1|        baixo|
|          1|        3|          4|        1|   5|   5|   2|   1|   2|   1|             576.7|        médio|
|          3|        1|          1|        0|   7|   6|   6|   6|   2|   2|            530.58|        médio|
|          2|      

In [145]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Inicia uma sessão Spark
spark = SparkSession.builder.appName("RandomForestExample").getOrCreate()

# Lista de colunas para usar como features
feature_cols = ['TP_COR_RACA', 'TP_ESCOLA', 'CO_UF_PROVA','TP_LINGUA', 'Q001', 'Q002', 'Q003', 'Q004', 'Q006', 'Q024', 'NU_NOTA_MEDIA']

# Indexação de strings para a coluna alvo
indexer = StringIndexer(inputCol='CLASSE_MEDIAS', outputCol='label')

# Assembler para transformar as colunas de feature em um vetor único
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

# Criação do modelo RandomForestClassifier
rf = RandomForestClassifier(labelCol='label', featuresCol='features', numTrees=100, maxDepth=10, seed=42)

# Criação do Pipeline
pipeline = Pipeline(stages=[indexer, assembler, rf])

# Dividindo os dados em conjunto de treino e teste
(train_data, test_data) = df_teste.randomSplit([0.8, 0.2], seed=42)

# Treinando o modelo
model = pipeline.fit(train_data)

# Fazendo previsões no conjunto de teste
predictions = model.transform(test_data)

# Avaliando o modelo
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f'Acurácia do modelo: {accuracy:.2f}')

# Encerra a sessão Spark
spark.stop()


Acurácia do modelo: 0.99


In [169]:
result_pdf.head()

,TP_COR_RACA,TP_ESCOLA,CO_UF_PROVA,TP_LINGUA,Q001,Q002,Q003,Q004,Q006,Q024,NU_NOTA_MEDIA,CLASSE_MEDIAS
0,2,1,4,1,5,6,1,4,2,1,558.24,médio
1,3,1,1,1,3,1,1,2,1,2,394.62,baixo
2,2,1,1,1,5,5,2,2,2,1,414.10,baixo
3,3,1,4,1,5,5,2,2,2,1,438.10,baixo
4,1,3,4,1,5,5,2,1,2,1,576.70,médio


In [173]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cálculo da média e classificação
result_pdf['CLASSE_MEDIAS'] = pd.cut(result_pdf['NU_NOTA_MEDIA'], bins=[0, 500, 700, float('inf')], labels=['baixo', 'médio', 'alto'])
result_pdf.drop('NU_NOTA_MEDIA', axis=1, inplace=True)
# Continue com o processamento após essa correção...
print(result_pdf.head())
result_pdf = result_pdf.dropna()
print(result_pdf['CLASSE_MEDIAS'].value_counts())
# Preparação dos dados para modelagem
X = result_pdf.drop('CLASSE_MEDIAS', axis=1)
y = result_pdf['CLASSE_MEDIAS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Avaliação do
# Modelagem usando Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Avaliação do
model.fit(X_train, y_train)

# Avaliação do modelo
y_pred = model.predict(X_test)

# Avaliação do
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}')

KeyError: 'NU_NOTA_MEDIA'

In [172]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

# Visualização da matriz de confusão
fig, ax = plt.subplots(figsize=(8, 8))  # Ajuste o tamanho conforme necessário
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.title('Matriz de Confusão')
plt.xlabel('Previsões')
plt.ylabel('Verdadeiros')

# Salve a imagem como png
plt.savefig('confusion_matrix.png')
plt.show()

C:\Users\mathe\AppData\Local\Temp\ipykernel_63024\3856552612.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
